In [1]:
import collections
import networkx as nx
import sys
import os
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [34]:
graph_collections, graph_name = read_graph("./test-graphs")
G = graph_collections[37]


{'ABILENE': {'index': 0, 'nodes': 11, 'edges': 14}, 'ABVT': {'index': 1, 'nodes': 23, 'edges': 31}, 'AGIS': {'index': 2, 'nodes': 25, 'edges': 30}, 'ARN.GRAPHML': {'index': 3, 'nodes': 30, 'edges': 29}, 'BELLCANADA': {'index': 4, 'nodes': 48, 'edges': 64}, 'BELLSOUTH.GRAPHML': {'index': 5, 'nodes': 51, 'edges': 66}, 'BICS.GRAPHML': {'index': 6, 'nodes': 33, 'edges': 48}, 'CERNET.GRAPHML': {'index': 7, 'nodes': 41, 'edges': 58}, 'CESNET200706.GRAPHML': {'index': 8, 'nodes': 44, 'edges': 51}, 'CHINANET.GRAPHML': {'index': 9, 'nodes': 42, 'edges': 66}, 'COGENTCO.GRAPHML': {'index': 10, 'nodes': 197, 'edges': 243}, 'COLT.GRAPHML': {'index': 11, 'nodes': 153, 'edges': 177}, 'DELTACOM.GRAPHML': {'index': 12, 'nodes': 113, 'edges': 161}, 'GARR201110.GRAPHML': {'index': 13, 'nodes': 59, 'edges': 74}, 'GEANT2001': {'index': 14, 'nodes': 27, 'edges': 38}, 'GRNET.GRAPHML': {'index': 15, 'nodes': 37, 'edges': 42}, 'INTELLIFIBER.GRAPHML': {'index': 16, 'nodes': 73, 'edges': 95}, 'INTEROUTE.GRAPHML'

In [35]:
# geo_loc_data = get_geo_data("Internet2LatLong.csv")
geo_loc_data = {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
                for (k, v) in G.nodes(data=True)}
# print(geo_loc_data)
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)


In [36]:

source_1, source_2, source_3, source_4, *others = eccentricity(G), load_centrality(G), betweenness_centrality(G),current_flow_betweenness_centrality(G), approximate_current_flow_betweenness_centrality(G)  # eigenvector_centrality(G),

# print(source_2)


In [37]:
maxi_mini_result = maxi_mini(source_1, source_2, source_3, source_4,no_of_evidences=4)
probability_weights_multi_res=probability_weights_multi(maxi_mini_result, e_1=source_1, e_2=source_2, e_3=source_3, e_4=source_4, number_of_evidences=4)
convert_dict_multi_results = convert_dict_multi(probability_weights_multi_res)


In [22]:
rank_result_multi(convert_dict_multi_results)


([],
 [('0', 0.0),
  ('2', 0.0),
  ('3', 0.0),
  ('4', 0.0),
  ('1', -0.18539280448294232),
  ('5', -0.18539280448294232),
  ('6', -0.18539280448294232),
  ('9', -0.18539280448294232),
  ('7', -0.37078560896588464),
  ('8', -0.37078560896588464),
  ('10', -0.37078560896588464)])

In [38]:
def rank_result_multi_test(*convert_dict_multi_results):
    """Generates ranked nodes according to predefined rankings.

    Returns:
        dict: dict of ranked nodes based on length (number) of evidences provided.
    """
    convert_dict_multi_results = list(*convert_dict_multi_results)

    def sorter(evidence_results_input):
        ranked_nodes = [{k: {'l': v['l'], 'h': v['h'], 'D_2SN': v['h']-v['l']} for k, v in x.items()}
                        for x in evidence_results_input]
        ranked_nodes = sorted([(k, v['D_2SN']) for x in ranked_nodes for k, v in x.items(
        )], key=lambda elem: elem[1], reverse=True)
        opti_rank = [(k, v) for k, v in ranked_nodes if v > 0]
        return opti_rank, ranked_nodes
    if len(convert_dict_multi_results) == 2:
        evidence_result_D_2SN = [{k: evidence_multi(v['h'], v['l'], v['t'], v2['h'], v2['l'], v2['t']) for k, v in x.items() for k2, v2 in y.items() if k2 == k}
                                 for x, y in zip(*convert_dict_multi_results[0:2])]
        return sorter(evidence_result_D_2SN)
    elif len(convert_dict_multi_results) == 3:
        evidence_result_D_2SN = [{k: evidence_multi(v['h'], v['l'], v['t'], v2['h'], v2['l'], v2['t'], v3['h'], v3['l'], v3['t'], no_of_evidences=3)
                                  for k, v in x.items() for k2, v2 in y.items() for k3, v3 in z.items() if all((k, k2, k3))}
                                 for x, y, z in zip(*convert_dict_multi_results[0:3])]
        return sorter(evidence_result_D_2SN)
    elif len(convert_dict_multi_results) == 4:
        evidence_result_D_2SN = [{k: evidence_multi(v['h'], v['l'], v['t'], v2['h'], v2['l'], v2['t'], v3['h'], v3['l'], v3['t'], v4['h'], v4['l'], v4['t'], no_of_evidences=4)
                                  for k, v in w.items() for k2, v2 in x.items() for k3, v3 in y.items() for k4, v4 in z.items() if all((k, k2, k3, k4))}
                                 for w,x,y,z in zip(*convert_dict_multi_results[0:4])]
        return sorter(evidence_result_D_2SN)


In [40]:
# [1,2,3,4,5,6][0:4]
# rank_result_multi_test(convert_dict_multi_results)

[1, 2, 3, 4]